In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.model_selection import GridSearchCV

In [2]:
# чтение данных
df = pd.read_csv('data.csv')
df.head()

,"Макс. ПДЗ за Y-1 год, дней","Сред. ПДЗ за Y-1 год, дней","Кол-во просрочек свыше 5-ти дней за Y-1 год, шт.","Общая сумма ПДЗ свыше 5-ти дней за Y-1 год, руб.","Кол-во раз ПДЗ за Y-1 год, шт.",Итого,"Y-4, Нематериальные активы, RUB","Y-3, Нематериальные активы, RUB","Y-2, Нематериальные активы, RUB","Y-1, Нематериальные активы, RUB",...,"Y-1, Прибыль (убыток) до налогообложения , RUB","Y-4, Прибыль (убыток) от продажи, RUB","Y-3, Прибыль (убыток) от продажи, RUB","Y-2, Прибыль (убыток) от продажи, RUB","Y-1, Прибыль (убыток) от продажи, RUB",Факт просрочки,Просрочка более 30 дней,Просрочка 0-30,"Оценка потенциала контрагента 1, руб.","Оценка потенциала контрагента 2, руб."
0,0,0.0,0,0.000000,0,10.0,2.895541e+06,6.245860e+06,9.050955e+06,9.885987e+06,...,3.603784e+09,3.280355e+09,6.200120e+09,8.716191e+08,3.658634e+09,1,0,1,-1.0,-1.0
1,0,0.0,0,0.000000,0,20.0,0.000000e+00,3.885350e+04,3.439490e+04,2.929936e+04,...,8.747516e+07,1.630064e+07,1.109172e+07,5.135732e+07,9.411019e+07,1,0,1,-1.0,-1.0
2,7,5.5,1,132825.299363,2,40.0,2.468153e+06,1.288025e+07,8.694904e+06,4.958599e+06,...,-6.456439e+08,4.148586e+08,1.611318e+08,-9.298981e+07,-1.207210e+08,1,0,1,-1.0,-1.0
3,0,0.0,0,0.000000,0,10.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,3.999298e+09,4.903117e+09,5.186553e+09,7.869977e+09,4.029232e+09,1,0,1,-1.0,-1.0
4,2,2.0,0,0.000000,2,20.0,5.503185e+05,5.210191e+05,4.490446e+05,3.987261e+05,...,4.960408e+10,2.338912e+10,3.727984e+10,5.307524e+10,5.622122e+10,1,0,1,-1.0,-1.0


In [3]:
df.shape

(853, 75)

## Исследование
* Проведем исследование подхода к моделированию на таргете "Факт просрочки"

### Выбор модели
* Так как полученный датафрейм не обладает большим количеством строк, для моделирования будем использовать модель логистической регрессии.

### Предобработка данных
* Предварительно нормализуем значения фичей для ускорения моделирования (отметим, что это не обязательный шаг, но так процесс схождения к минимуму выполнится быстрее)
* Так как в датасете присутствуют три таргета - для каждого построим отдельную модель

### Подбор параметров
* Подберем паракметры для логистической регрессии с помощью GridSearchCV

### Факт просрочки

In [4]:
y = df['Факт просрочки']
X = df.drop(['Факт просрочки', 'Просрочка более 30 дней', 'Просрочка 0-30'], axis=1)

scaler = StandardScaler()
transformer = ColumnTransformer([("st_scaler", 
                                 scaler, 
                                 X.columns)],
                                 remainder="passthrough")

transformed_X = transformer.fit_transform(X)

logreg = LogisticRegression(max_iter=10000, random_state=42)
param = {'C':[0.001, 0.01,  0.1, 1, 5], 
         'class_weight': [None, 'balanced'],
         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}
clf_for_fact_default = GridSearchCV(logreg,
                   param,
                   scoring='roc_auc',
                   cv=10)
clf_for_fact_default.fit(transformed_X,y)
print(f"Classifier best score - {clf_for_fact_default.best_score_}")
print(f"Classifier best parametrs - {clf_for_fact_default.best_params_}")

Classifier best score - 0.675996173945502
Classifier best parametrs - {'C': 1, 'class_weight': 'balanced', 'solver': 'saga'}


In [5]:
print('Факт просрочки')
print(df['Факт просрочки'].value_counts())
print('\nПросрочка 0-30')
print(df['Просрочка 0-30'].value_counts())
print('\nПросрочка более 30 дней')
print(df['Просрочка более 30 дней'].value_counts())

Факт просрочки
1    471
0    382
Name: Факт просрочки, dtype: int64

Просрочка 0-30
1    503
0    350
Name: Просрочка 0-30, dtype: int64

Просрочка более 30 дней
0    696
1    157
Name: Просрочка более 30 дней, dtype: int64


### Обучение и тестирование
* Отметим, что так как у нас несбалансированная выборка, мы будем использовать стратифицированную кросс-валидацию и метрику roc_auc_score для проверки качества


In [6]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_score = []
i = 1
for train_index, test_index in kf.split(transformed_X,y):
    print('{} из KFold {}'.format(i, kf.n_splits))
    X_train, X_test = transformed_X[train_index], transformed_X[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]

    lr = LogisticRegression(C=1, max_iter=10000, class_weight='balanced',
                   random_state=42, solver='saga')
    lr.fit(X_train, y_train)
    score = roc_auc_score(y_test, lr.predict_proba(X_test)[:, 1])
    print('ROC_AUC score:',score)
    cv_score.append(score) 
    i += 1
    
print(f"\nСредний ROC_AUC score: {np.mean(cv_score)}")

1 из KFold 5
ROC_AUC score: 0.7164819944598338
2 из KFold 5
ROC_AUC score: 0.6762917933130699
3 из KFold 5
ROC_AUC score: 0.6996407847471676
4 из KFold 5
ROC_AUC score: 0.7007278835386338
5 из KFold 5
ROC_AUC score: 0.6360582306830908

Средний ROC_AUC score: 0.6858401373483592


### Значимость фичей
* Оценим вклад, вносимый каждой из фичей в моделирование и удалим те, importance для которых меньше, чем 0.1

In [7]:
plt.figure(figsize=(20, 5))
logreg = LogisticRegression(C=1, solver='saga',
                            max_iter=10000, class_weight='balanced',random_state=42).fit(transformed_X, y)
importance = pd.DataFrame(logreg.coef_[0], X.columns)
importance.head()

,0
"Макс. ПДЗ за Y-1 год, дней",0.196017
"Сред. ПДЗ за Y-1 год, дней",0.285458
"Кол-во просрочек свыше 5-ти дней за Y-1 год, шт.",-0.116567
"Общая сумма ПДЗ свыше 5-ти дней за Y-1 год, руб.",-0.214554
"Кол-во раз ПДЗ за Y-1 год, шт.",0.579504


<Figure size 1440x360 with 0 Axes>

In [8]:
y = df['Факт просрочки']
X = df.drop(['Факт просрочки', 'Просрочка более 30 дней', 'Просрочка 0-30'], axis=1)

scaler = StandardScaler()
transformer = ColumnTransformer([("st_scaler", 
                                 scaler, 
                                 X.columns)],
                                 remainder="passthrough")

transformed_X = transformer.fit_transform(X)

logreg = LogisticRegression(C=1, solver='saga', max_iter=10000, random_state=42).fit(transformed_X, y)
importance = pd.DataFrame(logreg.coef_[0], X.columns)

for col in X.columns:
    if np.abs(importance.loc[col].values[0]) <= 0.1:
        X.drop(col, axis=1, inplace=True)

scaler = StandardScaler()
transformer = ColumnTransformer([("st_scaler", 
                                 scaler, 
                                 X.columns)],
                                 remainder="passthrough")

transformed_X = transformer.fit_transform(X)

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_score = []
i = 1
for train_index, test_index in kf.split(transformed_X,y):
    print('{} из KFold {}'.format(i, kf.n_splits))
    X_train, X_test = transformed_X[train_index], transformed_X[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]

    lr = LogisticRegression(C=1, max_iter=10000, class_weight='balanced', solver='saga',
                   random_state=42)
    lr.fit(X_train, y_train)
    y_predict = lr.predict_proba(X_test)[:, 1]
    score = roc_auc_score(y_test, y_predict)
    print('ROC_AUC score:',score)
    cv_score.append(score) 
    i += 1
    
print(f"\nСредний ROC_AUC score: {np.mean(cv_score)}")

1 из KFold 5
ROC_AUC score: 0.7256232686980609
2 из KFold 5
ROC_AUC score: 0.6892788063000829
3 из KFold 5
ROC_AUC score: 0.7050290135396519
4 из KFold 5
ROC_AUC score: 0.7057670772676373
5 из KFold 5
ROC_AUC score: 0.6409574468085106

Средний ROC_AUC score: 0.6933311225227887


### Threshold 
* Посмотрим на confusion_matrix: главная цель - определение "плохих" контрагентов (аналог антифрода). Поэтому нам нужно уменьшить ложно негативные значения (левый нижний угол матрицы), чтобы снизить риск классификации в реальности "плохого" контрагента как "хорошего".
* Уменьшение threshold дает нам уменьшение ложно негативных значений, но соответственно увеличивает ложно положительные (в реальных кейсах в таком случае возрастет нагрузка, например, на операторов, которым предстоит вручную проверять кейсы, в которых модель ложно отнесла контрагента к "плохому").

In [9]:
cf = confusion_matrix(y_test, (y_predict >= 0.4).astype(int))
cf

array([[25, 51],
       [10, 84]])

### Пайплайн
* Построим функцию-пайплайн проведенного выше обучения

In [10]:
def train_test_func_with_get_metric(df, model,
                                    num_splits=5,
                                    delete_non_important=True,
                                    threshold_importance = 0.1,
                                    target_column='Факт просрочки',
                                    cols_to_drop_from_X=['Факт просрочки', 'Просрочка более 30 дней', 'Просрочка 0-30'],
                                    threshold=0.5,
                                    random_state=42):
    '''
    Функция для обучения функции и проверки результатов обучения с помощью кросс-валидации.
    :return - лист с полученными метриками roc_auc_score
    :df - DataFrame,
    :model - используемая модель,
    :num_splits - на скольких сплитах проводить кросс-валидацию,
    :delete_non_important - True - если нужно удалить фичи, вклад которых меньше порога, False - не проводится измерение вклада,
    :threshold_importance - порог для удаления неважных фичей, 
    :cols_to_drop_from_X - фичи, которые нужно удалить из df,
    :threshold - порог принятия решения для классификации,
    :random_state - int
    '''
    
    # разделим данные на фичи и таргет
    X = df.drop(cols_to_drop_from_X, axis=1)
    y = df[target_column]
    
    metrics = []

    skf = StratifiedKFold(n_splits=num_splits, random_state=random_state, shuffle=True)
    i = 0
    
    # удаление не вносящих вклад в обучение фичей
    if delete_non_important:
        scaler = StandardScaler()
        transformer = ColumnTransformer([("st_scaler", 
                                 scaler, 
                                 X.columns)],
                                 remainder="passthrough")

        transformed_X = transformer.fit_transform(X)

        logreg = LogisticRegression(C=1, solver='saga', max_iter=10000, random_state=42).fit(transformed_X, y)
        importance = pd.DataFrame(logreg.coef_[0], X.columns)

        for col in X.columns:
            if np.abs(importance.loc[col].values[0]) <= threshold_importance:
                X.drop(col, axis=1, inplace=True)

    
    # стандартизация данных
    scaler = StandardScaler()
    transformer = ColumnTransformer([("st_scaler", 
                                      scaler, 
                                      X.columns)],
                                      remainder="passthrough")

    transformed_X = transformer.fit_transform(X)
            
    # кросс-валидация
    for train_index, test_index in skf.split(transformed_X, y):
        X_train, X_test = transformed_X[train_index], transformed_X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # обучение и прогноз
        lr = model
        lr.fit(X_train, y_train)
        y_predict = lr.predict_proba(X_test)[:, 1]
        metrics.append(roc_auc_score(y_test, y_predict))

        # полученные метрики
        print(f"K FOLD: {i+1}")
        print(f"AUC = {metrics[i]}")
        i += 1

    print(f"MEAN AUC = {np.mean(metrics)}")
    cf = confusion_matrix(y_test, (y_predict >= threshold).astype(int))
    return metrics, cf

In [11]:
# параметры подобранные с помощью GridSearch
clf_for_fact_default.best_params_

{'C': 1, 'class_weight': 'balanced', 'solver': 'saga'}

In [12]:
%%time

lr_for_fact_default = LogisticRegression(C=clf_for_fact_default.best_params_['C'], 
                                         max_iter=10000, 
                                         class_weight=clf_for_fact_default.best_params_['class_weight'],
                                         solver=clf_for_fact_default.best_params_['solver'],
                                         random_state=42)
metrics_for_lr_for_fact_default, classification_matrix_lr_for_fact_default = train_test_func_with_get_metric(df,
                                                                                                lr_for_fact_default)

K FOLD: 1
AUC = 0.7256232686980609
K FOLD: 2
AUC = 0.6892788063000829
K FOLD: 3
AUC = 0.7050290135396519
K FOLD: 4
AUC = 0.7057670772676373
K FOLD: 5
AUC = 0.6409574468085106
MEAN AUC = 0.6933311225227887
CPU times: user 7 s, sys: 52.4 ms, total: 7.06 s
Wall time: 4.21 s


In [13]:
%%time

lr_for_fact_default = LogisticRegression(C=clf_for_fact_default.best_params_['C'], 
                                         max_iter=10000, 
                                         class_weight=clf_for_fact_default.best_params_['class_weight'],
                                         solver=clf_for_fact_default.best_params_['solver'],
                                         random_state=42)
metrics_for_lr_for_fact_default, classification_matrix_lr_for_fact_default_0_4 = train_test_func_with_get_metric(df,
                                                                                                lr_for_fact_default,
                                                                                                threshold=0.4)

K FOLD: 1
AUC = 0.7256232686980609
K FOLD: 2
AUC = 0.6892788063000829
K FOLD: 3
AUC = 0.7050290135396519
K FOLD: 4
AUC = 0.7057670772676373
K FOLD: 5
AUC = 0.6409574468085106
MEAN AUC = 0.6933311225227887
CPU times: user 6.93 s, sys: 60.4 ms, total: 6.99 s
Wall time: 4.2 s


In [14]:
print('Classification matrix - threshold=0.5')
print(classification_matrix_lr_for_fact_default)
print('\nClassification matrix - threshold=0.4')
print(classification_matrix_lr_for_fact_default_0_4)

Classification matrix - threshold=0.5
[[55 21]
 [52 42]]

Classification matrix - threshold=0.4
[[25 51]
 [10 84]]


### Просрочка 0-30
* Подберем параметры, используя GridSearchCV
* Обучим, используя реализованный пайплайн

In [15]:
y = df['Просрочка 0-30']
X = df.drop(['Факт просрочки', 'Просрочка более 30 дней', 'Просрочка 0-30'], axis=1)

scaler = StandardScaler()
transformer = ColumnTransformer([("st_scaler", 
                                 scaler, 
                                 X.columns)],
                                 remainder="passthrough")

transformed_X = transformer.fit_transform(X)

logreg = LogisticRegression(max_iter=10000, random_state=42)
param = {'C':[0.001, 0.01,  0.1, 1, 5], 
         'class_weight': [None, 'balanced'],
         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}
clf_for_default_0_30 = GridSearchCV(logreg,
                   param,
                   scoring='roc_auc',
                   cv=10)
clf_for_default_0_30.fit(transformed_X,y)
print(f"Classifier best score - {clf_for_default_0_30.best_score_}")
print(f"Classifier best parametrs - {clf_for_default_0_30.best_params_}")

Classifier best score - 0.6803619047619047
Classifier best parametrs - {'C': 5, 'class_weight': 'balanced', 'solver': 'newton-cg'}


In [16]:
%%time

lr_default_0_30 = LogisticRegression(C=clf_for_default_0_30.best_params_['C'], 
                                         max_iter=10000, 
                                         class_weight=clf_for_default_0_30.best_params_['class_weight'],
                                         solver=clf_for_default_0_30.best_params_['solver'],
                                         random_state=42)
metrics_for_lr_default_0_30,classification_matrix_lr_default_0_30 = train_test_func_with_get_metric(df,
                                                                                                lr_default_0_30,
                                                                                        target_column='Просрочка 0-30')

K FOLD: 1
AUC = 0.7096181046676097
K FOLD: 2
AUC = 0.7284299858557284
K FOLD: 3
AUC = 0.7048090523338048
K FOLD: 4
AUC = 0.7985714285714284
K FOLD: 5
AUC = 0.7127142857142857
MEAN AUC = 0.7308285714285714
CPU times: user 1.92 s, sys: 14 ms, total: 1.94 s
Wall time: 1.13 s


In [17]:
classification_matrix_lr_default_0_30

array([[34, 36],
       [11, 89]])

### Просрочка более 30 дней

In [18]:
y = df['Просрочка более 30 дней']
X = df.drop(['Факт просрочки', 'Просрочка более 30 дней', 'Просрочка 0-30'], axis=1)

scaler = StandardScaler()
transformer = ColumnTransformer([("st_scaler", 
                                 scaler, 
                                 X.columns)],
                                 remainder="passthrough")

transformed_X = transformer.fit_transform(X)

logreg = LogisticRegression(max_iter=10000, random_state=42)
param = {'C':[0.001, 0.01,  0.1, 1, 5], 
         'class_weight': [None, 'balanced'],
         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}
clf_for_default_30 = GridSearchCV(logreg,
                   param,
                   scoring='roc_auc',
                   cv=10)
clf_for_default_30.fit(transformed_X,y)
print(f"Classifier best score - {clf_for_default_30.best_score_}")
print(f"Classifier best parametrs - {clf_for_default_30.best_params_}")

Classifier best score - 0.7635952380952381
Classifier best parametrs - {'C': 0.001, 'class_weight': 'balanced', 'solver': 'liblinear'}


In [19]:
%%time

lr_default_30 = LogisticRegression(C=clf_for_default_30.best_params_['C'], 
                                         max_iter=10000, 
                                         class_weight=clf_for_default_30.best_params_['class_weight'],
                                         solver=clf_for_default_30.best_params_['solver'],
                                         random_state=42)
metrics_for_lr_default_30,classification_matrix_lr_default_30 = train_test_func_with_get_metric(df,
                                                                                                lr_default_30,
                                                                                target_column='Просрочка более 30 дней')

K FOLD: 1
AUC = 0.7548387096774193
K FOLD: 2
AUC = 0.8230665467625898
K FOLD: 3
AUC = 0.7250449640287769
K FOLD: 4
AUC = 0.738222325365514
K FOLD: 5
AUC = 0.8110930610350429
MEAN AUC = 0.7704531213738686
CPU times: user 1.98 s, sys: 7.48 ms, total: 1.99 s
Wall time: 1.92 s


In [20]:
classification_matrix_lr_default_30

array([[127,  12],
       [ 12,  19]])